# **BERTopic on Italian, German and English on Amazon_massive_intent dataset**
In our earlier experiment with English news articles, the KeyBERT-inspired topic representations clearly outperformed basic frequency-based labels (c-TF-IDF), giving us cleaner and more meaningful descriptions.

We then tested the same approach on German and French Amazon reviews, which are much messier due to slang, typos, and strong opinions. We trained separate BERTopic models for each language and applied the KeyBERT-style method to see if we could still get clear product-related topics.However, the results were much weaker: both languages reached coherence scores of only 0.10–0.14, far below the ~0.40 we achieved on the BBC News dataset.

**In this notebook,** we test whether the same strategy works in a multilingual setting using the **amazon_massive_intent dataset** in Italian and German. This dataset is simpler and cleaner than Amazon Reviews, so we want to see if the model performs better when there is less linguistic noise.

We train separate BERTopic models for each language, just like we did with the Amazon Reviews experiments, and apply the KeyBERT-inspired representation to check whether we can extract clear and interpretable for **assistant command patterns**.


**Goal:** To assess whether BERTopic can consistently uncover similar **intent-related clusters** across three different languages Italian, German, English in the massive dataset, and to analyze how language-specific phrasing influences cluster structure.



In [1]:
!pip install --upgrade pip
!pip install bertopic datasets sentence_transformers pandas spacy scikit-learn
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm
!python -m spacy download it_core_news_sm
!python -m spacy download es_core_news_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 158.8 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 152.9 MB/s  0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting

In [ ]:
from datasets import load_dataset

ds_italian = load_dataset("SetFit/amazon_massive_intent_it-IT")
docs_train_italian = ds_italian["train"]["text"]
categories_train_italian = ds_italian["train"]["label_text"]


ds_de = load_dataset("SetFit/amazon_massive_intent_de-DE")
docs_train_de = ds_de["train"]["text"]
categories_train_de = ds_de["train"]["label_text"]

ds_english = load_dataset("SetFit/amazon_massive_intent_en-US")
docs_train_english = ds_english["train"]["text"]
categories_train_english = ds_english["train"]["label_text"]


# spanish dataset
ds_es = load_dataset("SetFit/amazon_massive_intent_es-ES")
docs_train_es = ds_es["train"]["text"]
categories_train_es = ds_es["train"]["label_text"]


### Data Preparation
We decided to use **lemmatization** this time to improve BERTopic. We did so because of the following reasons:
- we can remove the noise (words like Produkte, Produkten, Produktes will be merged into one -> Produkt)

- smaller vocabulary means more stable clustering

In [ ]:
import spacy

nlp_it = spacy.load("it_core_news_sm")

#lemmatization for Italian only
def lemmatize_it(text: str):
    doc = nlp_it(text)
    return [tok.lemma_ for tok in doc if tok.is_alpha]

#apply lemmatization to italian documents
lemmatized_it = [lemmatize_it(text) for text in docs_train_italian[:50000]]

#example
print("Original Italian:", docs_train_italian[38])
print("Lemmatized Italian:", lemmatized_it[38])


Original Italian: voglio del ramen da portare via olly hai qualche consiglio
Lemmatized Italian: ['volere', 'di il', 'ramen', 'da', 'portare', 'via', 'olly', 'avere', 'qualche', 'consiglio']


In [ ]:
import spacy


nlp_en = spacy.load("en_core_web_sm")

#lemmatization for English only
def lemmatize_en(text: str):
    doc = nlp_en(text)
    return [tok.lemma_ for tok in doc if tok.is_alpha]

#lemmatization to English documents
lemmatized_en = [lemmatize_en(text) for text in docs_train_english[:50000]]

#example
print("Original English:", docs_train_english[0])
print("Lemmatized English:", lemmatized_en[0])

Original English: wake me up at nine am on friday
Lemmatized English: ['wake', 'I', 'up', 'at', 'nine', 'am', 'on', 'friday']


In [ ]:
import spacy


nlp_de = spacy.load("de_core_news_sm")

#lemmatization for german only
def lemmatize_de(text: str):
    doc = nlp_de(text)
    return [tok.lemma_ for tok in doc if tok.is_alpha]

#lemmatization to English documents
lemmatized_de = [lemmatize_de(text) for text in docs_train_de[:50000]]

#example
print("Original German:", docs_train_de[0])
print("Lemmatized German:", lemmatized_de[0])

Original German: weck mich am freitag um neun uhr auf
Lemmatized German: ['Weck', 'mich', 'an', 'Freitag', 'um', 'neun', 'Uhr', 'auf']


In [ ]:
def remove_stopwords(lemmas: list[str], lang: str) -> str:
    stopwords_map = {
        "it": nlp_it.Defaults.stop_words,
        "de": nlp_de.Defaults.stop_words,
        "en": nlp_en.Defaults.stop_words

    }

    if lang not in stopwords_map:
        raise ValueError("lang must be 'it' or 'en'")

    stopwords = stopwords_map[lang]
    cleaned = (lemma for lemma in lemmas if lemma.lower() not in stopwords)
    return " ".join(cleaned)

docs_cleaned_italian = [remove_stopwords(lemmas, "it") for lemmas in lemmatized_it]
print("Cleaned Italian:", docs_cleaned_italian[38])

docs_cleaned_english = [remove_stopwords(lemmas, "en") for lemmas in lemmatized_en]
print("Cleaned English:", docs_cleaned_english[0])

docs_cleaned_de = [remove_stopwords(lemmas, "de") for lemmas in lemmatized_de]
print("Cleaned German:", docs_cleaned_de[0])

Cleaned Italian: volere di il ramen portare olly
Cleaned English: wake friday
Cleaned German: Weck Freitag


### **Set up**
##### **BERTopic model for Italian**

We trained a BERTopic model on the Amazon Massive intent dataset using different language embeddings e.g. Italian, German, English. Instead of restricting the model to 10 predefined topics, we allowed BERTopic to learn the topic structure organically, specifying only a minimum topic size of 30 documents.

In [ ]:
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

vectorizer_model = CountVectorizer(stop_words=None)

In [ ]:
representation_model_italian = KeyBERTInspired()

topic_model_baseline_italian = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="italian",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=30,
    nr_topics=None,
    representation_model=representation_model_italian
)

topics_base_italian, probs_base_italian = topic_model_baseline_italian.fit_transform(docs_train_italian)

2025-12-07 21:34:12,658 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/360 [00:00<?, ?it/s]

2025-12-07 21:43:18,053 - BERTopic - Embedding - Completed ✓
2025-12-07 21:43:18,054 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 21:43:28,758 - BERTopic - Dimensionality - Completed ✓
2025-12-07 21:43:28,762 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 21:43:32,835 - BERTopic - Cluster - Completed ✓
2025-12-07 21:43:32,844 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 21:44:43,077 - BERTopic - Representation - Completed ✓


##### **BERTopic model for German**

We repeated the same setup for the German dataset; however, instead of using the Amazon multilingual review dataset, we used  the Amazon MASSIVE intent dataset_german and followed the same procedure.

In [ ]:
representation_model_de = KeyBERTInspired()

topic_model_baseline_de = BERTopic(
    embedding_model=sentence_model,
    vectorizer_model=vectorizer_model,
    language="german",
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=30,
    nr_topics=None,
    representation_model=representation_model_de
)

topics_de, probs_de = topic_model_baseline_de.fit_transform(docs_train_de)

2025-12-07 21:44:59,985 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/360 [00:00<?, ?it/s]

2025-12-07 21:53:43,761 - BERTopic - Embedding - Completed ✓
2025-12-07 21:53:43,763 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 21:53:54,750 - BERTopic - Dimensionality - Completed ✓
2025-12-07 21:53:54,751 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 21:53:58,109 - BERTopic - Cluster - Completed ✓
2025-12-07 21:53:58,118 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 21:55:06,447 - BERTopic - Representation - Completed ✓


In [ ]:
# A topic must contain at least 20 documents.If a cluster has fewer than 20 documents, BERTopic will merge it into a larger, more similar topic.
# nr_topics — target number of topics after reduction

# nr_topics=10 means:

# After clustering, BERTopic will reduce the topics to exactly 10.So if it initially finds 30 topics, it will merge them until only 10 remain.

##### **BERTopic model for English**

We repeat the same setup for the English Dataset.

In [ ]:
representation_model_en = KeyBERTInspired()

topic_model_baseline_en = BERTopic(
    embedding_model=sentence_model,      # multilingual embedding model (same across datasets)
    vectorizer_model=vectorizer_model,   # same vectorizer model for consistent preprocessing
    language="english",                  # specify English tokenizer
    calculate_probabilities=True,
    verbose=True,
    min_topic_size=30,
    nr_topics=None,                      # let BERTopic learn the number of clusters
    representation_model=representation_model_en
)

topics_en, probs_en = topic_model_baseline_en.fit_transform(docs_train_english)

2025-12-07 21:55:21,538 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/360 [00:00<?, ?it/s]

2025-12-07 22:03:11,351 - BERTopic - Embedding - Completed ✓
2025-12-07 22:03:11,353 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-12-07 22:03:23,377 - BERTopic - Dimensionality - Completed ✓
2025-12-07 22:03:23,380 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-12-07 22:03:28,601 - BERTopic - Cluster - Completed ✓
2025-12-07 22:03:28,610 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-12-07 22:04:42,398 - BERTopic - Representation - Completed ✓


### **Inspecting computed topics for Italian**

In [ ]:
info_df = topic_model_baseline_italian.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(8))

,Topic,Count,Top Keywords
1,0,904,"sono, favore, lette, se, chiocciola, alla"
2,1,797,"mettere, metti, favore, brani, riprodurre, stelle"
3,2,377,"spegni, elimina, ventuno, mettere, dimmi, rimuovere"
4,3,355,"dammi, dimmi, adesso, sono, lì, sarà"
5,4,330,"fissa, prima, dell, per, sera, stasera"
6,5,300,"mese, vai, favore, vado, settimana, appuntamento"
7,6,264,"fammi, darmi, tienimi, fornitore, olly, della"
8,7,244,"sono, dimmi, partita, momento, ora, sull"


The model is able to identify some meaningful intent-related themes in the Italian MASSIVE dataset. For example topic-1 showed somewhat clearer structure, capturing media-playback actions through verbs like **“mettere”** and **“metti”** (put/play) and nouns such as **“brani”** (songs). On the other hand, BERTopic in italian dataset produced several mixed or noisy clusters, indicating difficulty in extracting clean intent categories from short Italian utterances. However, the output also highlights key limitations. For instance, Topic 0 was dominated by extremely frequent conversational tokens such as **“sono”** (I am), **“favore”** (please), and even **“chiocciola”** (the @ symbol).

Together, these examples show that BERTopic struggles to form clean intent clusters when input texts are short, formulaic, and dominated by high-frequency filler words.

### **Inspecting computed topics for English**

We repeat the same steps for the inspection of the topics generated by the French model by building a clean topic overview table with interpretable keyword representations.

In [ ]:
info_df = topic_model_baseline_en.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(8))

,Topic,Count,Top Keywords
1,0,441,"darken, lighting, darker, lamp, brighten, lights"
2,1,407,"brighton, sunset, olly, warm, temperature, maine"
3,2,310,"clean, erase, empty, dinner, remove, clear"
4,3,288,"again, music, on, pause, too, audio"
5,4,247,"capacity, live, sapp, works, is, find"
6,5,220,"most, kim, elvis, miley, when, chan"
7,6,218,"list, please, amazon, any, freds, my"
8,7,208,"most, today, todays, let, now, notify"


As predicted, the English BERTopic model performs noticeably better than the Italian and German versions, producing cleaner and more semantically consistent clusters. For example, Topic 0 captures a coherent lighting-control intent with keywords such as **“darken,”** **“lighting,”** **“lamp,”** **“brighten,”** and **“lights”**, which aligns well with typical smart-home commands.

However, some clusters still suffer from noise and mixed semantics. Topic 5, for instance, blends celebrity names **(“kim,” “elvis,” “miley”) **with unrelated temporal words **(“when”)**, indicating that BERTopic occasionally forms clusters based on accidental lexical co-occurrence rather than true intent similarity

### **Inspecting computed topics for German**

We repeat the same steps for the inspection of the topics generated by the French model by building a clean topic overview table with interpretable keyword representations.

In [ ]:
info_df = topic_model_baseline_de.get_topic_info()
clean_df = info_df[info_df["Topic"] != -1].copy()

def clean_keywords(repr_list):
    return ", ".join(repr_list[:6])

clean_df["Top Keywords"] = clean_df["Representation"].apply(clean_keywords)

display_table = clean_df[["Topic", "Count", "Top Keywords"]]

import pandas as pd
pd.set_option('display.max_colwidth', None)
display(display_table.head(8))

,Topic,Count,Top Keywords
1,0,834,"bitte, auf, möchte, gerade, spielt, der"
2,1,833,"posteingang, füge, bekommen, öffne, erhalten, schicke"
3,2,704,"ausmachen, neuste, mir, wiedergabe, wurde, finde"
4,3,430,"auf, schalten, brauche, ausmachen, aller, bitte"
5,4,385,"prüfe, brauche, weckalarm, bestätige, tech, stunde"
6,5,329,"bitte, heutige, schön, gerade, samstag, tage"
7,6,214,"sprich, dich, bitte, kopfhörer, machen, sehr"
8,7,183,"spiel, spielen, komma, einschalten, auf, spiele"


The German BERTopic model shows mixed performance.
On the positive side, one cluster cleanly captures email-related actions, with keywords like in topic 2 **“posteingang”** (inbox), **“öffne” **(open), **“erhalten”** (receive), and **“schicke”** (send), indicating that the model successfully extracts a coherent communication-intent topic.


However, a clear limitation appears with the repeated use of the politeness word “bitte.” It shows up in Topic 0, Topic 3, Topic 5, and Topic 6, even though these topics correspond to different intents such as media playback, device control, daily schedule etc. Because
“bitte”is so common in short German commands, BERTopic overweights it, which causes clusters to mix polite filler words with the actual intent-related verbs and it reduced the overall clarity of the topics.

### Visualization

#### **Visualization of Italian topic space**

Similar to the previous notebook, we visualize the topics in a 2D projection to assess how well-separated they are.We did in three different language italian, english, german for amazon massive intent dataset. The accompanying bar chart allows us to inspect the key words within each cluster along with their relative importance.

In [ ]:
fig_map = topic_model_baseline_italian.visualize_topics()
fig_map.show()

fig_bar = topic_model_baseline_italian.visualize_barchart(top_n_topics=10)
fig_bar.show()

#### **Visualization of the English topic space**

In [ ]:
fig_map = topic_model_baseline_en.visualize_topics()
fig_map.show()

fig_bar = topic_model_baseline_en.visualize_barchart(top_n_topics=10)
fig_bar.show()

#### **Visualization of the German topic space**

In [ ]:
fig_map = topic_model_baseline_de.visualize_topics()
fig_map.show()

fig_bar = topic_model_baseline_de.visualize_barchart(top_n_topics=10)
fig_bar.show()

### Evaluation

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import itertools
from math import log

class TopicModelWrapper:
    def __init__(self, topics):
        self.topics = topics

    def get_topics(self):
        return self.topics

def evaluate_bertopic_pmi(
    topic_model,
    docs,
    top_k_coherence: int = 5,
    top_k_diversity: int = 10,
    skip_outlier: bool = True,
    tag: str = "Baseline",
):

    # 1) Extract topic -> list of topic words from BERTopic
    raw_topics = topic_model.get_topics()

    new_keywords = {
        topic_id: [word for word, _ in word_scores]
        for topic_id, word_scores in raw_topics.items()
    }

    # optional: drop outlier topic -1 for evaluation
    if skip_outlier and -1 in new_keywords:
        new_keywords = {tid: kws for tid, kws in new_keywords.items() if tid != -1}

    # 2) Build co-occurrence stats over docs (only topic words)
    # 2.1 Collect all unique keywords from topics
    all_keywords = sorted({w for kws in new_keywords.values() for w in kws})

    vectorizer = CountVectorizer(vocabulary=all_keywords, lowercase=True)
    X = vectorizer.fit_transform(docs)  # shape: (n_docs, n_terms)

    n_docs, n_terms = X.shape

    X_bin = (X > 0).astype(int)

    word_doc_counts = np.asarray(X_bin.sum(axis=0)).ravel()
    p_w = word_doc_counts / n_docs

    cooc_counts = (X_bin.T @ X_bin).toarray()
    p_ij = cooc_counts / n_docs

    vocab = np.array(vectorizer.get_feature_names_out())
    word2id = {w: i for i, w in enumerate(vocab)}

    # 3) Define NPMI helpers and compute topic coherence
    def npmi_pair(w1, w2):
        i = word2id.get(w1)
        j = word2id.get(w2)
        if i is None or j is None:
            return None

        pij = p_ij[i, j]
        if pij == 0:
            return None  # never co-occur

        pi = p_w[i]
        pj = p_w[j]

        pmi = log(pij / (pi * pj))
        return pmi / (-log(pij))

    def topic_npmi_coherence(topic_words, top_k=None):
        if top_k is not None:
            topic_words = topic_words[:top_k]

        scores = []
        for w1, w2 in itertools.combinations(topic_words, 2):
            score = npmi_pair(w1, w2)
            if score is not None:
                scores.append(score)

        if not scores:
            return float("nan")
        return float(np.mean(scores))

    topic_scores = {
        topic_id: topic_npmi_coherence(words, top_k=top_k_coherence)
        for topic_id, words in new_keywords.items()
    }

    coherence_df = pd.DataFrame(
        {
            "Topic": list(topic_scores.keys()),
            "NPMI": list(topic_scores.values()),
        }
    ).sort_values("Topic")

    mean_npmi = float(np.nanmean(coherence_df["NPMI"]))

    # 4) Topic diversity
    def topic_diversity(topics_dict, k=10):
        # collect top-k words for each topic
        topk_words = []
        for tid, words in topics_dict.items():
            topk_words.extend(words[:k])

        if not topk_words:
            return float("nan")

        unique_words = set(topk_words)
        T = len(topics_dict)
        total_words = T * k

        return len(unique_words) / total_words

    diversity = float(topic_diversity(new_keywords, k=top_k_diversity))

    print(f"{tag} Model - NPMI: {mean_npmi:.4f}")
    print(f"{tag} Model - Diversity: {diversity:.4f}")

    return coherence_df, mean_npmi, diversity

In [ ]:
#from calculate_t_coherence_and_diversity import evaluate_bertopic_pmi

_, npm_german, div_german = evaluate_bertopic_pmi(
    topic_model=topic_model_baseline_de,
    docs=docs_cleaned_de,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="German"
)

_, npm_italian, div_italian = evaluate_bertopic_pmi(
    topic_model=topic_model_baseline_italian,
    docs=docs_cleaned_italian,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="Italian"
)
_, npm_en, div_en = evaluate_bertopic_pmi(
    topic_model=topic_model_baseline_en,
    docs=docs_cleaned_english,
    top_k_coherence=10,
    top_k_diversity=25,
    tag="English"
)

results_df = pd.DataFrame({
    "Metric": ["Coherence (NPMI)", "Diversity"],
    "German": [npm_german, div_german],
    "Italian": [npm_italian, div_italian],
    "English": [npm_en, div_en],
})
display(results_df)

German Model - NPMI: 0.2941
German Model - Diversity: 0.2106
Italian Model - NPMI: 0.1873
Italian Model - Diversity: 0.1723
English Model - NPMI: 0.1933
English Model - Diversity: 0.1886


,Metric,German,Italian,English
0,Coherence (NPMI),0.294115,0.187320,0.193270
1,Diversity,0.210556,0.172346,0.188642


# Evaluation

The goal of this project was to replicate the success observed in our English news experiment and, to some extent, in the more challenging multilingual Amazon Reviews dataset, **using a simpler dataset to evaluate whether the model would perform better.** Additionally, we sought to compare how the Italian language behaves relative to the others. Previously, we experimented with the German Amazon multilingual review dataset, but the data proved too noisy due to its nature as a review corpus.

Although we were able to extract intent-related themes for Italian, German, and English, the quantitative metrics indicate that the results were again considerably less successful. The coherence scores for German and Italian were 0.29 and 0.18, respectively. We expected the English portion of the Amazon MASSIVE Intent dataset to perform better; however, it produced a coherence score of 0.19, which is also very low. In contrast, the BBC News dataset achieved a coherence score of 0.40, highlighting the performance gap. Although the coherence score for the German dataset increased slightly from 0.26 to 0.29, this improvement is minimal.

With the Amazon Reviews dataset, we initially hypothesized that the model underperformed due to the nature of review data—texts that are short, informal, and heavily saturated with sentiment (e.g., “bad,” “perfect”), which complicates topic extraction. However, the Amazon MASSIVE Intent dataset is simple and clean, without slang, typos, or strong sentiment. Despite this, the model still performed poorly.

These results suggest that the multilingual BERTopic model may still have notable limitations, particularly when applied to intent-based datasets across different languages.On the other hand, the availability of suitable, high-quality datasets in multiple languages is limited, which further constrains our ability to comprehensively evaluate BERTopic’s multilingual performance.